## TEMPLATE

In [ ]:
  const url =
    "https://utfs.io/f/877538c9-48c4-4561-87e1-a1cf737dcd5f-v5ud9p.pdf";
  const response = await fetch(url);
  const blob = await response.blob();
  const loader = new WebPDFLoader(blob);
  const docs = await loader.load();

## BASIC TEMPLATE

In [ ]:
  const textSplitter = new RecursiveCharacterTextSplitter({
    chunkSize: 1000,
    chunkOverlap: 200,
  });
  const splits = await textSplitter.splitDocuments(docs);
  const vectorStore = await MemoryVectorStore.fromDocuments(
    splits,
    new OpenAIEmbeddings({
      openAIApiKey: process.env.OPENAI_API_KEY,
    })
  );

  //   const retriever = vectorStore.asRetriever();
  //   const prompt = await pull<ChatPromptTemplate>("rlm/rag-prompt");
  //   const llm = new ChatOpenAI({
  //     model: "gpt-3.5-turbo",
  //     temperature: 0,
  //     openAIApiKey: process.env.OPENAI_API_KEY,
  //   });

  //   const declarativeRagChain = RunnableSequence.from([
  //     {
  //       context: retriever.pipe(formatDocumentsAsString),
  //       question: new RunnablePassthrough(),
  //     },
  //     prompt,
  //     llm,
  //     new StringOutputParser(),
  //   ]);

  //   const res = await declarativeRagChain.invoke("What is Amarok?");

  //   console.log(res);
  const retriever = vectorStore.asRetriever({ k: 3, searchType: "similarity" });
  const retrievedDocs = await retriever.invoke(
    "What is Amarok?"
  );
  console.log(retrievedDocs.length);
  console.log(retrievedDocs[0].pageContent);

## CUSTOM TEMPLATE

import { PromptTemplate } from "@langchain/core/prompts";
import { createStuffDocumentsChain } from "langchain/chains/combine_documents";

const template = `Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:`;

const customRagPrompt = PromptTemplate.fromTemplate(template);

const ragChain = await createStuffDocumentsChain({
  llm,
  prompt: customRagPrompt,
  outputParser: new StringOutputParser(),
});
const context = await retriever.getRelevantDocuments(
  "what is task decomposition"
);

await ragChain.invoke({
  question: "What is Task Decomposition?",
  context,
});